In [ ]:
# Importing necessary packages
import torch
import math
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms, models
import torch.nn.functional as F
import time
import os
import torch.backends.cudnn as cudnn
import torchvision.models as models
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import torchvision
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# Initializing parameters
os.environ["CUDA_VISIBLE_DEVICES"] = '0'                # GPU Number 
start_time = time.time()
batch_size = 40
learning_rate = 0.001
num_of_epochs = 15
root_dir = 'C/Users/MYDocuments/Deep Learning Project/ashar_vgg19'
default_directory = 'C/Users/MYDocuments/Deep Learning Project/ashar_vgg19'
val_losses = []
train_losses = []
#

In [ ]:
# Defining Data Augmentation
transform_train = transforms.Compose([
    transforms.Resize(size=(224, 224)),               # Random Position Crop
    transforms.RandomHorizontalFlip(),                  # right and left flip
    transforms.ToTensor(),                              # change [0,255] Int value to [0,1] Float value
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010) )  # RGB Normalize Standard Deviation
])

transform_test = transforms.Compose([
    transforms.Resize(size=(224, 224)),                               
    transforms.ToTensor(),                              # change [0,255] Int value to [0,1] Float value
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010) )  # RGB Normalize Standard Deviation
])

#

In [ ]:
 # Automatic downloading datasets and data augmentation
train_dataset = datasets.CIFAR10(root=root_dir,
                                 train=True,
                                 transform=transform_train,
                                 download=True)

test_dataset = datasets.CIFAR10(root=root_dir,
                                train=False,
                                transform=transform_test)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,            # at Training Procedure, Data Shuffle = True
                                           num_workers=4)           # CPU loader number

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,            # at Test Procedure, Data Shuffle = False
                                          num_workers=4)            # CPU loader number




In [ ]:
# Setting up the processing device and initializing the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def initialize_model( num_classes, use_pretrained=True):

    model_ft = models.vgg19_bn(pretrained=use_pretrained)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
    input_size = 224
    return model_ft, input_size

# Initialize the model for this run
model, input_size = initialize_model(10, use_pretrained=True)
'''
#loading Pretrainined VGG
model= models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, 10)

'''


In [ ]:
# Defining optimizer
model = model.to(device)
#optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
optimizer = optim.Adagrad(model.parameters(), lr=0.01, lr_decay=0, weight_decay=0, initial_accumulator_value=0, eps=1e-10) #Changed the optimizer to Adagrad 


criterion = nn.CrossEntropyLoss()

if torch.cuda.device_count() > 0:
    print("USE", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model).cuda()
    cudnn.benchmark = True
else:
    print("USE ONLY CPU!")



In [ ]:
# Defining the train function
def train(epoch):
    model.train()
    train_loss = 0 
    total = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = Variable(data.cuda()), Variable(target.cuda())
        else:
            data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        writer.add_scalar('Loss/train', train_loss, epoch)
        
        total += target.size(0)
        correct += predicted.eq(target.data).cpu().sum()
        if batch_idx % 10 == 0:
            print('Epoch: {} | Batch_idx: {} |  Loss: ({:.4f}) | Acc: ({:.2f}%) ({}/{})'
                  .format(epoch, batch_idx, train_loss / (batch_idx + 1), 100. * correct / total, correct, total))
        
        TRAIN_LOSS.append(train_loss)
        TRAIN_ACC.append((100. * correct / total))

            
    dict = {'train loss' : TRAIN_LOSS, 'train accuracy' : (100. * correct / total)}
    df = pd.DataFrame(dict)
    df.to_csv('train_loss_and_acc_vgg.csv')
        



In [ ]:
# Defining the test function
def test():
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        if torch.cuda.is_available():
            data, target = Variable(data.cuda()), Variable(target.cuda())
        else:
            data, target = Variable(data), Variable(target)

        outputs = model(data)
        loss = criterion(outputs, target)

        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        writer.add_scalar('Loss/val', test_loss, epoch)
        
        total += target.size(0)
        correct += predicted.eq(target.data).cpu().sum()
        
        TEST_LOSS.append(test_loss)
        TEST_ACC.append((100. * correct / total))
        
    print('# TEST : Loss: ({:.4f}) | Acc: ({:.2f}%) ({}/{})'
          .format(test_loss / (batch_idx + 1), 100. * correct / total, correct, total))


            
    dict = {'test loss' : TEST_LOSS, 'test accuracy' : (100. * correct / total)}
    df = pd.DataFrame(dict)
    df.to_csv('test_loss_and_acc_vgg.csv')


def save_checkpoint(directory, state, filename='latest.tar.gz'):

    if not os.path.exists(directory):
        os.makedirs(directory)

    model_filename = os.path.join(directory, filename)
    torch.save(state, model_filename)
    print("=> saving checkpoint")

def load_checkpoint(directory, filename='latest.tar.gz'):

    model_filename = os.path.join(directory, filename)
    if os.path.exists(model_filename):
        print("=> loading checkpoint")
        state = torch.load(model_filename)
        return state
    else:
        return None

In [ ]:
# Main function for training the model

TRAIN_LOSS = []
TEST_LOSS = []
TRAIN_ACC = []
TEST_ACC = []

start_epoch = 0

checkpoint = load_checkpoint(default_directory)
if not checkpoint:
    pass
else:
    start_epoch = checkpoint['epoch'] + 1
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

writer = SummaryWriter(root_dir)    

for epoch in range(start_epoch, num_of_epochs):

    if epoch < 20:
        lr = learning_rate
    elif epoch < 40:
        lr = learning_rate * 0.1
    else:
        lr = learning_rate * 0.01
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    train(epoch)
    save_checkpoint(default_directory, {
        'epoch': epoch,
        'model': model,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
    })
    test()  

writer.close()
    
now = time.gmtime(time.time() - start_time)
print('{} hours {} mins {} secs for training'.format(now.tm_hour, now.tm_min, now.tm_sec))